In [1]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install imutils


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import cv2 
import numpy as np
from imutils import paths

In [5]:
from sklearn.preprocessing import LabelBinarizer

In [6]:
datapath = r'D:\Vacasion_Upgrade_Skills\VideoClassification\vieoclassifier_data'
outputmodel = r'D:\Vacasion_Upgrade_Skills\VideoClassification'
outputlabelbinaries = r'D:\Vacasion_Upgrade_Skills\VideoClassification\vieoclassifier_data\videoclassificationbinaties'

In [7]:
sports_labels = set(['boxing', 'swimming', 'table_tennis'])
print("Images is being loaded....")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in sports_labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    labels.append(label)


print("Images loaded successfully!")

Images is being loaded....
Images loaded successfully!


In [8]:
data = np.array(data)
# This converts the list of labels into a NumPy array.
labels = np.array(labels)

# hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
(x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42) 

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
traininAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype='float32' )
traininAugmentation.mean = mean
validationAugmentation.mean = mean

In [13]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout 
from tensorflow.keras import Model


In [14]:
baseModel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
headModel = baseModel.output

headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
for baseModelLayers in baseModel.layers:
    baseModelLayers.trainable = False


In [15]:
from tensorflow.keras.optimizers import SGD
# stocastic gradient descent


In [16]:
epoch = 25
opt = SGD(learning_rate=0.0001, momentum=0.9, decay=1e-4/epoch)

c:\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [17]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# since we have catagories.

history = model.fit(
    traininAugmentation.flow(x_train, y_train, batch_size=10),
    steps_per_epoch=len(x_train) // 10,
    validation_data=validationAugmentation.flow(x_test, y_test),
    validation_steps=len(x_test) // 10,
    epochs=epoch
    )

Epoch 1/25


c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


156/156 ━━━━━━━━━━━━━━━━━━━━ 112s 678ms/step - accuracy: 0.4971 - loss: 1.2244 - val_accuracy: 0.9178 - val_loss: 0.2443
Epoch 2/25


c:\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


156/156 ━━━━━━━━━━━━━━━━━━━━ 27s 172ms/step - accuracy: 1.0000 - loss: 0.1617 - val_accuracy: 0.9178 - val_loss: 0.2433
Epoch 3/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 109s 695ms/step - accuracy: 0.8242 - loss: 0.4263 - val_accuracy: 0.9465 - val_loss: 0.1627
Epoch 4/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 26s 164ms/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 0.9465 - val_loss: 0.1633
Epoch 5/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 110s 702ms/step - accuracy: 0.9029 - loss: 0.2673 - val_accuracy: 0.9522 - val_loss: 0.1310
Epoch 6/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 26s 165ms/step - accuracy: 0.8000 - loss: 0.3609 - val_accuracy: 0.9522 - val_loss: 0.1309
Epoch 7/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 109s 696ms/step - accuracy: 0.9017 - loss: 0.2533 - val_accuracy: 0.9541 - val_loss: 0.1202
Epoch 8/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 26s 163ms/step - accuracy: 1.0000 - loss: 0.0865 - val_accuracy: 0.9503 - val_loss: 0.1199
Epoch 9/25
156/156 ━━━━━━━━━━━━━━━━━━━━ 106s 677ms/step - accuracy: 0.9266 - loss: 0.2026 -